# GLM Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
import statsmodels.api as sm
import seaborn as sns

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pickle


## Model Maker Class

In [2]:
class aviation_data_model:
    def __init__(self, x, y, drop_categories = None, test_size = 0.3, random_state = 2023):
        self.x = x.copy()
        self.drop_categories = drop_categories
        self.y = y
        self.random_state = random_state
        self.test_size = test_size
        self.model = None
        self.x_oh = None
        self.x_con = None
        self.results = None
        self.pvalues = None
        self.X_test = None
        self.Y_test = None

        
    def one_hot(self):
        #Check to see if dummy parameter passed correctly
        if type(self.drop_categories[0]) == tuple:
            
            categories = []
            for i, j in self.drop_categories:
                # Collect subset of columns to dummify
                categories.append(i)
                
                # If a specific drop value has been given for a column, add a 0- to it so it is 'first' and drops accordingly
                if j != 'first':
                    self.x[i] = np.where(self.x[i] == j, '0-' + j, self.x[i])
        else:
            raise TypeError('Invalid OneHotEncoder values. Use None or a list of tuples of (column name, category name')
            
        self.x_oh = pd.get_dummies(data= self.x, columns = categories, drop_first=True)
        # Log line - uncomment below to check shapes after important steps
        # print(f"OneHot Resulting Shape: {self.x_oh.shape}")
        return self.x_oh
    
    def train_test(self, x, y):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=self.test_size, random_state=self.random_state)
        return (X_train, X_test, y_train, y_test)
        
    
    def fit_binomial(self, link = 'logit'):
        # Start with OneHotEncoding if given
        if self.drop_categories:
            model_x = self.one_hot()
        else:
            model_x = self.x
            
        # Add column for constant per statsmodels GLM requirements
        x_con = sm.add_constant(model_x[list(model_x.columns)])
        self.x_con = x_con
        
        # Log line - uncomment below to check shapes after important steps
        # print(f"After constant Resulting Shape: {self.x_con.shape}")
        
        #Train Test Split
        X_train, X_test, y_train, y_test = self.train_test(x_con, self.y)

        
        # Log lines - uncomment below to check shapes after important steps
        # print(f"After tts X_train Resulting Shape: {X_train.shape}")
        # print(f"After tts X_test Resulting Shape: {X_test.shape}")
        # print(f"After tts y_train Resulting Shape: {y_train.shape}")
        # print(f"After tts y_test Resulting Shape: {y_test.shape}")
        
        if link == 'logit':
            link_function = sm.families.links.Logit()
        elif link == 'probit':
            link_function = sm.families.links.Probit()
        
        #Make and fit a model
        glm_bin = sm.GLM(
            y_train,
            X_train,
            family=sm.families.Binomial(link=link_function)).fit()
        
        #Stash results as accessible attributes
        self.results = glm_bin.summary()
        self.model = glm_bin
        self.pvalues = np.round(glm_bin.pvalues, 6).to_frame().sort_values(by = 0)
        self.pvalues.rename({0: 'p_values'}, axis =1 , inplace=True)
        self.X_test = X_test
        self.y_test = y_test

In [3]:
def glm_accuracy(model, x, y):
    test_pred = model.predict(x)
    test_pred = test_pred.map(lambda x: 1 if x >=0.50 else 0)
    acc = pd.concat([test_pred, y], axis = 1)
    return acc[acc.iloc[:, 0] == acc.iloc[:, 1]].shape[0] / y.shape[0]

In [4]:
def is_top_model(data, model, top_x):
    top_list = list(data['model'].value_counts()[:top_x + 1].index.str.upper())
    return True if model.upper() in top_list else False

In [5]:
def is_top_make(data, make, top_x):
    top_list = list(data['make'].value_counts()[:top_x + 1].index.str.upper())
    return True if make.upper() in top_list else False

In [6]:
def is_top_family(data, family, top_x):
    top_list = list(data['aircraft_family'].value_counts()[:top_x + 1].index.str.upper())
    return True if family.upper() in top_list else False

In [7]:
def is_top_city(data, city, top_x):
    top_list = list(data['city'].value_counts()[:top_x + 1].index.str.upper())
    return True if city.upper() in top_list else False

In [8]:
# def bundler(val, top_n):
#     top_list = list(data.value_counts()[:top_n + 1].index.str.upper())
#     new_data = np.where(data.str.upper().isin(top_list), data.str.upper(), 'UNCOMMON' )
#     return new_data

### Read in data

In [9]:
df = pd.read_csv("../datasets/data_cleaned/alaska_single_engine_clean.csv")

## Binarize, bundle, and engineer

Group 'UNK' and 'Unknown'

In [10]:
df['purpose_of_flight'] = df['purpose_of_flight'].map(lambda x: 'UNK' if x=='Unknown' else x)

In [11]:
df['scheduled'] = df['scheduled'].map(lambda x: 'UNK' if x == 'Unknown' else x)

The following lines are designed to help balance out categorical values, since there are many infrequent occurrences as well as very common occurrences. They set a cutoff in terms of the most frequent values, and anything outside of that gets lumped together as some variant of "Uncommon"

In [12]:
df['model'] = [x.upper() if is_top_model(df, x,200) else 'UNCOMMON MODEL' for x in df['model']]

In [13]:
df['make'] = [x.upper() if is_top_make(df, x,50) else 'UNCOMMON MAKE' for x in df['make']]


In [14]:
df['city'] = [x.upper() if is_top_city(df, x,75) else 'UNCOMMON CITY' for x in df['city']]

In [15]:
df['aircraft_category'].value_counts()

aircraft_category
AIR         5271
HELI         303
ULTR           5
WSFT           4
GYRO           3
GLI            2
UNMANNED       1
PPAR           1
Name: count, dtype: int64

### Drop everything that isn't a airplane or helicoper

In [16]:
df.drop( df[~df['aircraft_category'].isin(['AIR', 'HELI'])].index, inplace = True)

## Manual GLM Attempt with multiple features

### Create X

In [17]:
X_1a = df.drop(['ntsb_no', 'probable_cause', 'airport_name', 'event_type', 'mkey', 'city', 'n', 'has_safety_rec', 'report_type', 'highest_injury_level', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'airport_id', 'far', 'aircraft_damage', 'operator', 'event_year', 'event_season', 'event_day', 'aircraft_category', 'has_injury', 'event_time','has_aircraft_damage', 'is_accident', 'make_model', 'has_fatal_injury', 'aircraft_family', 'event_time_of_day', 'event_am_pm'], axis=1)

In [18]:
X_1a.head()

,latitude,longitude,make,model,amateur_built,scheduled,purpose_of_flight,weather_condition,event_month,event_hour,occurred_near_airport
0,61.775160,-152.15263,CESSNA,UNCOMMON MODEL,0,UNK,PERS,Unknown,9,8,0
1,60.805019,-161.78648,PIPER,PA-18-150,0,UNK,PERS,Unknown,9,12,0
2,59.646929,-151.49323,CESSNA,A185F,0,UNK,BUS,VMC,9,13,1
3,64.267579,-147.68704,HELIO,H-295,0,NSCH,BUS,Unknown,9,16,0
4,64.650753,-149.83639,UNCOMMON MAKE,UNCOMMON MODEL,1,UNK,PERS,VMC,9,15,0


### Create Y

In [19]:
df['highest_injury_level'].value_counts()

highest_injury_level
None Reported    4000
Minor             625
Fatal             573
Serious           376
Name: count, dtype: int64

In [20]:
df['highest_injury_level'] = np.where(
    df['highest_injury_level'].str.contains('None Reported') |
    df['highest_injury_level'].str.contains('Minor'),
    0,
    1)

In [21]:
y = df['highest_injury_level']
# y = df['has_fatal_injury']

### Create dummies

In [22]:
X_1a_dummy = pd.get_dummies(data=X_1a,
                         columns=['make',
                                  'model',
                                  'scheduled',
                                  'purpose_of_flight',
                                  'weather_condition',
                                  'event_month',
                                  'event_hour'
                                 ],
                         drop_first=True
        )

In [23]:
X_1a_dummy.shape

(5574, 311)

### Add constant for use in GLM

In [24]:
X_1a_con = sm.add_constant(X_1a_dummy[list(X_1a_dummy.columns)])

In [25]:
X_1a_con.shape

(5574, 312)

### Train test split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_1a_con, y, random_state=2023)

In [27]:
X_train.shape

(4180, 312)

In [28]:
glm_bin = sm.GLM(
    y_train,
    X_train,
    family=sm.families.Binomial(link=sm.families.links.Logit())).fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
glm_bin_results = glm_bin.summary()

### Check accuracy

In [ ]:
glm_accuracy(glm_bin, X_test, y_test)

In [ ]:
1 - y.mean()

### Result: Accuracy does technically outperform baseline - p-values can be trusted

In [ ]:
glm_bin_pvalues = np.round(glm_bin.pvalues, 4).to_frame().sort_values(by =  0, ascending = True)

In [ ]:
glm_bin_coef_df = glm_bin.params.to_frame().sort_values(by = 0, ascending = False)

In [ ]:
glm_bin_pvalues.rename({0: 'p_value'}, axis = 1, inplace=True)

In [ ]:
glm_bin_coef_df.rename({0: 'coef'}, axis = 1, inplace=True)

In [ ]:
glm_bin_pvalues.head(30)

In [ ]:
# glm_bin_pvalues.to_csv('../results/glm_pvalues_manyfeatures_severe_and_fatal_y.csv')

## Trying again with modified predictor variables (notably aircraft family and city) using aviation_model class

In [ ]:
X_1b = df.drop(['ntsb_no', 'probable_cause', 'airport_name', 'event_type', 'mkey', 'n', 'has_safety_rec', 'report_type', 'highest_injury_level', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'airport_id', 'far', 'aircraft_damage', 'operator', 'event_year', 'event_season', 'event_day', 'aircraft_category', 'has_injury', 'event_time','has_aircraft_damage', 'is_accident', 'make_model', 'make', 'model', 'has_fatal_injury', 'event_time_of_day', 'event_am_pm'], axis=1)

In [ ]:
# y = df['highest_injury_level']
y = df['has_fatal_injury']

In [ ]:
X_1b.columns

In [ ]:
glm1z = aviation_data_model(X_1b, y, [
    ('city', 'first'),
    ('scheduled', 'UNK'),
    ('purpose_of_flight', 'UNK'),
    ('weather_condition', 'Unknown'),
    ('event_month', 'first'),
    ('event_hour', 'first'),
    ('aircraft_family', 'first')
], test_size=0.2)

In [ ]:
glm1z.fit_binomial()

In [ ]:
y.mean()

In [ ]:
glm_accuracy(glm1z.model, glm1z.X_test, glm1z.y_test)

In [ ]:
glm1z.pvalues.head(30)

## Third attempt introducing time_of_day

In [ ]:
X_1c = df.drop(['ntsb_no', 'probable_cause', 'airport_name', 'event_type', 'mkey', 'n', 'has_safety_rec', 'report_type', 'highest_injury_level', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'airport_id', 'far', 'aircraft_damage', 'operator', 'event_year', 'event_season', 'event_day', 'aircraft_category', 'has_injury', 'event_time','has_aircraft_damage', 'is_accident', 'make_model', 'make', 'model', 'has_fatal_injury', 'event_hour', 'event_am_pm'], axis=1)

In [ ]:
y = df['highest_injury_level']
# y = df['has_fatal_injury']

In [ ]:
X_1c.columns

In [ ]:
glm1y = aviation_data_model(X_1c, y, [
    ('city', 'first'),
    ('scheduled', 'UNK'),
    ('purpose_of_flight', 'UNK'),
    ('weather_condition', 'Unknown'),
    ('event_month', 'first'),
    ('event_time_of_day', 'first'),
    ('aircraft_family', 'first')
], test_size=0.2)

In [ ]:
glm1y.fit_binomial()

In [ ]:
1 - y.mean()

In [ ]:
glm_accuracy(glm1y.model, glm1y.X_test, glm1y.y_test)

In [ ]:
glm1y.pvalues.head(30)

In [ ]:
# glm1y.pvalues.to_csv('../results/glm_pvalues_largefeatureset3_severe_and_fatal_y_acc836.csv')

In [ ]:
y_pred_proba = glm1y.model.predict(glm1y.X_test)

In [ ]:
y_pred = np.where(y_pred_proba >= 0.5, 1, 0)

## Fourth attempt swapping time_of_day to AM/PM

In [ ]:
X_1d = df.drop(['ntsb_no', 'probable_cause', 'airport_name', 'event_type', 'mkey', 'n', 'has_safety_rec', 'report_type', 'highest_injury_level', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'airport_id', 'far', 'aircraft_damage', 'operator', 'event_year', 'event_season', 'event_day', 'aircraft_category', 'has_injury', 'event_time','has_aircraft_damage', 'is_accident', 'make_model', 'make', 'model', 'has_fatal_injury', 'event_hour', 'event_time_of_day'], axis=1)

In [ ]:
y = df['highest_injury_level']
#y = df['has_fatal_injury']

In [ ]:
X_1d.columns

In [ ]:
glm1p = aviation_data_model(X_1d, y, [
    ('city', 'first'),
    ('scheduled', 'UNK'),
    ('purpose_of_flight', 'UNK'),
    ('weather_condition', 'Unknown'),
    ('event_month', 'first'),
    ('event_am_pm', 'first'),
    ('aircraft_family', 'first')
], test_size=0.2)

In [ ]:
glm1p.fit_binomial()

In [ ]:
1 - y.mean()

In [ ]:
glm_accuracy(glm1p.model, glm1p.X_test, glm1p.y_test)

In [ ]:
glm1p.pvalues.head(30)

### Confusion Matrix Check In

In [ ]:
# glm1p.pvalues.to_csv('../results/glm_pvalues_largefeatureset4_severe_and_fatal_y_acc838.csv')

In [ ]:
y_pred_proba = glm1p.model.predict(glm1p.X_test)

In [ ]:
y_pred = np.where(y_pred_proba >= 0.5, 1, 0)

In [ ]:
    def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
      
        matrix = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10,7))
        sns.heatmap(matrix, annot=True, fmt='g', cmap="Blues", 
                    xticklabels=['Predicted No Fatalities', 'Predicted Fatalities'], 
                    yticklabels=['Actually No Fatalities', 'Actually Had Fatalities'])
        plt.title(title)
        plt.show()

In [ ]:
plot_confusion_matrix(glm1z.y_test, y_pred)

## Rapid Fire P Values

## 1 - Weather Condition

In [ ]:
y = df['highest_injury_level']

In [ ]:
X = df[['weather_condition']]

In [ ]:
X.dtypes

In [ ]:
y.value_counts()

In [ ]:
# X['purpose_of_flight'].value_counts()

In [ ]:
glm1a = aviation_data_model(X, y,[
    ('weather_condition', 'Unknown')
])

In [ ]:
glm1a.fit_binomial()

In [ ]:
# glm1a.pvalues.to_csv('../results/glm_pvalues_weather_severe_and_fatal_y.csv')

In [ ]:
np.exp(glm1a.model.params)

## 2 - Aircraft Family

In [ ]:
X_2 = df[['aircraft_family']]

In [ ]:
y = df['highest_injury_level']

In [ ]:
glm2a = aviation_data_model(X_2, y, drop_categories=[('aircraft_family', 'first')])

In [ ]:
glm2a.fit_binomial()

In [ ]:
glm2a_pvalues = glm2a.pvalues

In [ ]:
glm2a_pvalues.head(10)

In [ ]:
# glm2a_pvalues.to_csv('../results/glm_pvalues_aircraftfamily_severe_and_fatal_y.csv')

## 3 - Latitude and Longitude

In [ ]:
X_3 = df[['latitude', 'longitude']]

In [ ]:
y = df['highest_injury_level']

In [ ]:
glm3a = aviation_data_model(X_3, y)

In [ ]:
X_3.dtypes

In [ ]:
glm3a.fit_binomial()

In [ ]:
glm3a.pvalues

## 4 - City

In [ ]:
X_4 = df[['city']]

In [ ]:
y = df['highest_injury_level']

In [ ]:
glm4a = aviation_data_model(X_4, y, [
    ('city', 'first')
])

In [ ]:
glm4a.fit_binomial()

In [ ]:
glm4a.pvalues

In [ ]:
1 - y.mean()

In [ ]:
glm_accuracy(glm4a.model, glm4a.X_test, glm4a.y_test)

## Try out p values again but with modified y (any injury)

## 5 - Weather Condition

In [ ]:
y = df['has_injury']

In [ ]:
X_5 = df[['weather_condition']]

In [ ]:
X.dtypes

In [ ]:
y.value_counts()

In [ ]:
# X['purpose_of_flight'].value_counts()

In [ ]:
glm5a = aviation_data_model(X_5, y,[
    ('weather_condition', 'VMC')
])

In [ ]:
glm5a.fit_binomial()

In [ ]:
glm5a.pvalues

In [ ]:
# glm1a.pvalues.to_csv('../results/glm_pvalues_weather_severe_and_fatal_y.csv')

In [ ]:
np.exp(glm5a.model.params)

In [ ]:
glm_accuracy(glm5a.model, glm5a.X_test, glm5a.y_test)

## 6 - Aircraft Family

In [ ]:
X_6 = df[['aircraft_family']]

In [ ]:
y = df['has_injury']

In [ ]:
glm6a = aviation_data_model(X_6, y, drop_categories=[('aircraft_family', 'first')])

In [ ]:
glm6a.fit_binomial()

In [ ]:
glm6a_pvalues = glm6a.pvalues

In [ ]:
glm6a_pvalues.head(10)

In [ ]:
# glm2a_pvalues.to_csv('../results/glm_pvalues_aircraftfamily_severe_and_fatal_y.csv')

In [ ]:
glm_accuracy(glm6a.model, glm6a.X_test, glm6a.y_test)

## 7 - Latitude and Longitude

In [ ]:
X_7 = df[['latitude', 'longitude']]

In [ ]:
y = df['has_injury']

In [ ]:
glm7a = aviation_data_model(X_7, y)

In [ ]:
X_7.dtypes

In [ ]:
glm7a.fit_binomial()

In [ ]:
glm7a.pvalues

In [ ]:
glm_accuracy(glm7a.model, glm7a.X_test, glm7a.y_test)

## 8 - City

In [ ]:
X_8 = df[['city']]

In [ ]:
y = df['has_injury']

In [ ]:
glm8a = aviation_data_model(X_8, y, [
    ('city', 'first')
])

In [ ]:
glm8a.fit_binomial()

In [ ]:
glm8a.pvalues

In [ ]:
glm_accuracy(glm8a.model, glm8a.X_test, glm8a.y_test)